In [ ]:
## Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']
gaze_tasks = ['GazeCenter','GazeLeft','GazeRight']
attend_tasks = ['AttendFix', 'AttendBar']
# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

### Compute averaged decoding position and height over runs 

In [2]:
for subject in subjects:
    tsv_dir = '{}/{}/decode/tsv'.format(pp_dir, subject)
    try: os.makedirs(tsv_dir)
    except: pass

    for gaze_task in gaze_tasks:
        for attend_task in attend_tasks:
            for roi_num, roi in enumerate(rois):

                # load files
                run1_fn = "{}/{}/decode/fit/{}_ses-01_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)
                run2_fn = "{}/{}/decode/fit/{}_ses-02_task-{}{}_roi-{}_nvoxels-250_resizefactor-3_mcmcbarpars.tsv".format(pp_dir, subject, subject, gaze_task, attend_task, roi)

                run1_df = pd.read_csv(run1_fn,sep="\t")
                run2_df = pd.read_csv(run2_fn,sep="\t")

                tr = np.nanmean((run1_df.groupby(['time']).time.mean(),run2_df.groupby(['time']).time.mean()),axis=0)
                
                # create dataframe
                df_roi = pd.DataFrame({'subject': [subject] * tr.shape[0],
                                       'roi': [roi] * tr.shape[0],
                                       'TR': tr,
                                       'decoded_x_mean': np.nanmean((run1_df.groupby(['time']).x.mean(),run2_df.groupby(['time']).x.mean()),axis=0),
                                       'decoded_x_std': np.nanmean((run1_df.groupby(['time']).x.std(),run2_df.groupby(['time']).x.std()),axis=0),
                                       'decoded_h_mean': np.nanmean((run1_df.groupby(['time']).height.mean(),run2_df.groupby(['time']).height.mean()),axis=0),
                                       'decoded_h_std': np.nanmean((run1_df.groupby(['time']).height.std(),run2_df.groupby(['time']).height.std()),axis=0)})
                                       
                # across roi
                if roi_num > 0: df = pd.concat([df,df_roi], ignore_index=True)
                else: df = df_roi
                
            # Save dataframe as tsv
            df_fn = "{}/{}_task-{}{}_decode_par.pkl".format(tsv_dir,subject,gaze_task,attend_task)
            print('saving {}'.format(df_fn))
            df.to_pickle(df_fn)
            
            del df

saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-004/decode/tsv/sub-004_task-GazeRightAttendFix_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-004/decode/tsv/sub-004_task-GazeRightAttendBar_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/decode/tsv/sub-005_task-GazeCenterAttendFix_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/decode/tsv/sub-005_task-GazeCenterAttendBar_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/decode/tsv/sub-005_task-GazeLeftAttendFix_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/decode/tsv/sub-005_task-GazeLeftAttendBar_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/decode/tsv/sub-005_task-GazeRightAttendFix_decode_par.pkl
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp